In [1]:
with open('wikipedia_eng.txt','r',encoding='utf-8') as f:
    text=f.read()

In [2]:
print("Length: ",len(text))

Length:  215106987


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]^_`abcdefghijklmnopqrstuvwxyz{|}~
95


In [4]:
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[72, 73, 73, 1, 84, 72, 69, 82, 69]
hii there


In [5]:
# import tiktoken

In [6]:
# tokenizer=tiktoken.get_encoding("cl100k_base")
# vocab_size=100000
# tokens=tokenizer.encode("hello im rajat")
# print(tokens)
# decode=tokenizer.decode(tokens)
# print(decode)

In [7]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.type)
print(data[:10])

torch.Size([215106987]) <built-in method type of Tensor object at 0x000001D7AB965370>
tensor([34, 80, 82, 73, 76,  1, 73, 83,  1, 84])


Split into Train/Test

In [8]:
n=int(0.9*len(data))
train_data=data[:n]
test_data=data[n:]

Hyperparameters

In [9]:
torch.manual_seed(1234)
batch_size=32
block_size=8
max_iter=3000
eval_interval=300
learning_rate=1e-2
device='cuda' if torch.cuda.is_available() else 'cpu'
test_iter=200

Split into batches

In [10]:
def batchxy(split):
    if split=='train':
        data=train_data
    else:
        data=test_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    xl=[]
    yl=[]
    for i in ix:
        xl.append(data[i:i+block_size])
        yl.append(data[i+1:i+1+block_size])
    x=torch.stack(xl)
    y=torch.stack(yl)
    x,y=x.to(device),y.to(device)
    return x,y    

In [11]:
xb,yb=batchxy('train')
print(xb.shape)
print(xb)
print(yb.shape)
print(yb)

torch.Size([32, 8])
tensor([[73, 78, 71,  1, 53, 72, 69,  1],
        [88,  1, 78,  1, 31,  1, 76,  1],
        [66, 83, 73, 84, 69, 83,  0, 54],
        [69, 86, 79, 76, 85, 84, 73, 79],
        [23, 18, 23, 20, 10,  0,  1, 45],
        [76, 65, 67, 69,  1, 73, 78,  1],
        [76, 68, 15,  1, 42, 78,  1, 19],
        [ 1, 76, 69, 84, 84, 69, 82, 83],
        [77, 90, 65,  1, 65, 78, 68,  1],
        [ 1, 76, 73, 84, 84, 76, 69,  1],
        [84, 69, 82, 13,  1, 65, 67, 84],
        [84, 69, 82,  1, 73, 78, 67, 76],
        [68,  1, 65, 84,  1, 84, 72, 69],
        [92, 20, 21, 92, 92, 19, 17, 92],
        [84, 72, 73, 79, 80, 73, 65, 78],
        [ 1, 66, 65, 83, 75, 69, 84, 66],
        [84, 90, 69, 82, 76, 65, 78, 68],
        [78, 73, 90, 65, 84, 73, 79, 78],
        [82, 79, 82,  8, 83,  1, 36, 85],
        [ 1, 35, 82, 79, 75, 69, 78,  1],
        [78, 68,  1, 80, 72, 73, 76, 65],
        [65, 84, 72, 69, 82, 13,  1, 44],
        [84, 73, 86, 73, 84, 89,  0, 34],
        [66, 6

Bigram

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1234)

In [13]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','test']:
        losses=torch.zeros(test_iter)
        for k in range(test_iter):
            X,Y=batchxy(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [14]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
        66
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [15]:
model=BigramLanguageModel(vocab_size)
m=model.to(device)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1, 1), dtype=torch.long,device=device)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 95])
tensor(5.0978, grad_fn=<NllLossBackward0>)

$mzG>9n*=Is2DfNZ{F&.bK!|s'vbhOWjv"_P#6;>h3>h$L&cD)B=4A@Pt9R@+()f3PWiHAK{lb@-NpCIXT2N3P+,g[0-k,y*bK]~


Optimizer

In [16]:
optimizer=torch.optim.AdamW(m.parameters(),learning_rate)

In [17]:
for steps in range(max_iter):
    if steps%eval_interval==0:
        losses=estimate_loss()
        print(f"step {steps}: train loss {losses['train']:.4f},val loss {losses['test']:.4f}")
    xb,yb=batchxy('train')
    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

step 0: train loss 5.1475,val loss 5.1395
step 300: train loss 3.0677,val loss 3.0805
step 600: train loss 2.7295,val loss 2.7477
step 900: train loss 2.6664,val loss 2.6881
step 1200: train loss 2.6435,val loss 2.6573
step 1500: train loss 2.6223,val loss 2.6470
step 1800: train loss 2.6266,val loss 2.6393
step 2100: train loss 2.6069,val loss 2.6349
step 2400: train loss 2.6167,val loss 2.6324
step 2700: train loss 2.6173,val loss 2.6331
2.6320106983184814

Same tynkin f wilome an Ir Inm tansy Catecouthe cithing tom Tha Pige 15)

Tolsere trbellde Pasppeans
